In [1]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-huggingface
!pip install pypdf
!pip install transformers
!pip install faiss-gpu
!pip install gradio
!pip install langchain-nvidia-ai-endpoints
!pip install sentence-transformers
!pip install pymupdf==1.24.10
!pip install streamlit==1.38.0
!pip install python-pptx==1.0.2
!pip install Pillow==10.4.0
!pip install requests==2.32.3
!pip install frontend==0.0.3
!pip install llama-index-core==0.10.58
!pip install llama-index-readers-file==0.1.30
!pip install llama-index-llms-nvidia==0.1.4
!pip install llama-index-embeddings-nvidia==0.1.4
!pip install llama-index-vector-stores-milvus==0.1.20
!pip install pymilvus==2.4.4
!pip install -r https://raw.githubusercontent.com/nvidia-riva/python-clients/main/requirements.txt
!pip install --force-reinstall git+https://github.com/nvidia-riva/python-clients.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━

  Cloning https://github.com/nvidia-riva/python-clients.git to /tmp/pip-req-build-p39mn_h3
  Running command git clone --filter=blob:none --quiet https://github.com/nvidia-riva/python-clients.git /tmp/pip-req-build-p39mn_h3
  Resolved https://github.com/nvidia-riva/python-clients.git to commit f7d5e3e3dc8e139a015843cef495191b5061d073
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached grpcio-1.67.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
  Using cached grpcio_tools-1.67.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached grpcio-1.67.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
Using cached grpcio_tools-1.67.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2

In [2]:
import os
import tarfile
import torch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import faiss
import warnings
warnings.filterwarnings('ignore')
import json
import re
from langchain.schema import Document
import gradio as gr
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import gradio as gr
import subprocess
import os
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

# Step 2: Create the destination directory if it doesn't exist
import os
folder_path = '/content/drive/My Drive/Colab Notebooks/rita' ###Replace with path to the submissioon folder in your drive
os.makedirs(folder_path, exist_ok=True)
os.environ["NVIDIA_API_KEY"] = "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" ### Insert API Key

Mounted at /content/drive


In [4]:
print(folder_path)

# Verify the file exists
tar_path = os.path.join(folder_path, 'nemo_curator_env.tar.gz')

# Define the target directory for extraction
extract_dir = folder_path

# Extract the file
with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(extract_dir)

print(f"Extracted NeMo Curator environment to {extract_dir}")

/content/drive/My Drive/Colab Notebooks/rita
Extracted NeMo Curator environment to /content/drive/My Drive/Colab Notebooks/rita


In [17]:
import os
import json
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Path to the JSON file containing the extracted PDF content
folder_path = '/content/drive/My Drive/Colab Notebooks/rita/docextract'
scraped_data_file = os.path.join(folder_path, 'iphone_16_manual_extracted.json')

def load_scraped_data(file_path):
    """Load scraped data from a JSON file."""
    with open(file_path, 'r') as file:
        return json.load(file)

def clean_text(raw_text):
    """Clean up the text by removing unnecessary whitespace and formatting."""
    cleaned_text = re.sub(r'\s+', ' ', raw_text)  # Replace multiple whitespace with a single space
    return cleaned_text.strip()  # Remove leading and trailing whitespace

def chunk_text(text, chunk_size=500):
    """Chunk the text into smaller parts of specified size."""
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

def process_scraped_data(scraped_data):
    """Process scraped data to separate main content and metadata."""
    processed_docs = []

    for page_data in scraped_data:  # Iterate over each page's data
        page_number = page_data['page']
        content = page_data['text']
        device = page_data['device']

        # Clean the content and create metadata for each page
        main_content = clean_text(content)
        metadata = {
            'page_number': page_number,
            'device': device  # Add device name to metadata
        }

        # Chunk the cleaned content
        chunks = chunk_text(main_content)

        # Append chunks along with metadata
        for chunk in chunks:
            processed_docs.append({
                'text': chunk,
                'metadata': metadata
            })

    return processed_docs

def split_text(documents, chunk_size=200, chunk_overlap=50):
    """Split documents into chunks while preserving metadata."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    split_docs = []
    for doc in documents:
        # Split the main content and retain the metadata
        chunks = text_splitter.split_text(doc['text'])

        # Create a list of chunked documents with metadata
        for chunk in chunks:
            split_docs.append({
                'text': chunk,
                'metadata': doc['metadata']
            })

    return split_docs

# Load the extracted PDF content
scraped_data = load_scraped_data(scraped_data_file)

# Print the type and first few elements (or entire structure) of the scraped data
print(f"Type of scraped_data: {type(scraped_data)}")
print(f"First item of scraped_data: {scraped_data[0] if scraped_data else 'No data available'}")

# Process the scraped data
processed_documents = process_scraped_data(scraped_data)

# Now, split the processed documents into smaller chunks
chunked_documents = split_text(processed_documents)

# Output the chunked documents for verification
print(f"Total chunks created: {len(chunked_documents)}")
print(f"Sample chunk: {chunked_documents[0] if chunked_documents else 'No chunks available'}")

chunked_documents


Type of scraped_data: <class 'list'>
First item of scraped_data: {'text': 'Helpful? Yes NoTable of Contents \nSetup basics\niPhone has an easy setup process to help you get going when you first turn on your device. Whether\nyouʼre just getting started or want to make sure you have the basics set up, youʼre in the right place.\nTransfer your apps and info onto\nyour new iPhone\nWhen you first turn on your iPhone and begin the setup\nprocess, you can wirelessly transfer many of your apps,\nsettings, and content from your previous iPhone directly to your\nnew iPhone—just bring the two devices close together, then\nfollow the onscreen instructions.\nSet up cellular service and connect\nto Wi-Fi\nDepending on your model, use an eSIM from your carrier or\ninstall a physical SIM card to connect to your cellular network.\nTo check if your iPhone is connected to your cellular plan, go to\nSettings\xa0\xa0> Cellular.\nTo connect your iPhone to your home Wi-Fi network, go to\nSettings\xa0\xa0> Wi

[{'text': 'Helpful? Yes NoTable of Contents Setup basics iPhone has an easy setup process to help you get going when you first turn on your device. Whether youʼre just getting started or want to make sure you',
  'metadata': {'page_number': 1, 'device': 'iPhone 16'}},
 {'text': 'just getting started or want to make sure you have the basics set up, youʼre in the right place. Transfer your apps and info onto your new iPhone When you first turn on your iPhone and begin the',
  'metadata': {'page_number': 1, 'device': 'iPhone 16'}},
 {'text': 'When you first turn on your iPhone and begin the setup process, you can wirelessly transfer many of your apps, settings, and content from your previous iPhone directly to your new iPhone—just bring',
  'metadata': {'page_number': 1, 'device': 'iPhone 16'}},
 {'text': 'iPhone directly to your new iPhone—just bring the',
  'metadata': {'page_number': 1, 'device': 'iPhone 16'}},
 {'text': 'two devices close together, then follow the onscreen instruction

In [18]:
def download_embedding_model(model_name):
    global embeddings
    # making GPU is available
    device = "cuda:1" if torch.cuda.is_available() else "cpu"
    # pick the embedding model from huggingface
    encode_kwargs = {
                        "device": device,
                        "normalize_embeddings": True
                    }
    model_kwargs =  {
                        "device": device,
                        "trust_remote_code":True
                    }
    # Create a custom HuggingFaceEmbeddings instance
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

In [28]:
# Define global variables for embeddings and model names
vectorstore = None
embeddings = None
model_name = "Alibaba-NLP/gte-large-en-v1.5"
vectorDB_name = "./emanuals.db"

def initialize_embeddings():
    """Initialize the embedding model with remote code trust enabled."""
    global embeddings
    embeddings = HuggingFaceEmbeddings()

def create_vector_database(chunked_documents):
    """Create a Vector Database Base (VDB) from pre-processed document chunks."""
    global vectorstore
    try:
        # Ensure each chunked document has `text` and `metadata`
        wrapped_documents = []
        for doc in chunked_documents:
            # Verify doc structure
            if 'text' not in doc or 'metadata' not in doc:
                raise ValueError("Each chunked document must contain 'text' and 'metadata' keys.")
            # Wrap in Document object
            wrapped_documents.append(Document(page_content=doc['text'], metadata=doc['metadata']))

        # Create FAISS index and vector embeddings for chunks of data
        vectorstore = FAISS.from_documents(wrapped_documents, embeddings)

        # Save the vector database index locally
        vectorstore.save_local(vectorDB_name)

        return "Successfully created the vector database from chunked documents."
    except Exception as e:
        return f"Error creating vector database: {str(e)}"

# Initialize embeddings model
initialize_embeddings()

# Example usage
# Assuming `chunked_documents` is ready and structured correctly
result_message = create_vector_database(chunked_documents[1:10])
print(result_message)

Successfully created the vector database from chunked documents.


In [29]:
def load_vector_database(vectorDB_name):
    global vectorstore, embeddings
    try:
        vectorstore = FAISS.load_local(vectorDB_name, embeddings, allow_dangerous_deserialization=True)
        return "Successfully loaded the vector database."
    except Exception as e:
        return f"Error loading vector database: {str(e)}"

def test_vdb():
    global vectorstore, embeddings
    retriever = vectorstore.as_retriever()
    query = "How to power on my remote"
    results = retriever.get_relevant_documents(query)
    print(f"Number of retrieved documents: {len(results)}")
    for doc in results:
        print(doc.page_content[:200])  # Print first 100 characters of each documen

In [30]:
load_vector_database(vectorDB_name)
test_vdb()

Number of retrieved documents: 4
two devices close together, then follow the onscreen instructions. Set up cellular service and connect to Wi-Fi Depending on your model, use an eSIM from your carrier or install a physical SIM card to
later) or another iPhone (with iOS 17 or later), you can simply bring your iPhone near your other device. Otherwise, you can manually sign in. To verify that youʼre signed in, go to Settings > [yo
your carrier or install a physical SIM card to connect to your cellular network. To check if your iPhone is connected to your cellular plan, go to Settings > Cellular. To connect your iPhone to your
ouʼre home. Sign in to your Apple Account You use your Apple Account to access Apple services such as the App Store, the iTunes Store, Apple Books, Apple Music, FaceTime, iCloud, iMessage, and more.


In [31]:
def prompt_template():
    prompt = ChatPromptTemplate.from_template("""
    You are a helpful, patient, and kind assistant, specialized in assisting users with technology-related queries.
    Below are some guidelines for handling different types of user questions:

    General Approach:
    - Always prioritize simplicity in language, avoiding jargon or technical terms whenever possible.
    - Respond as concisely and briefly as possible, using short sentences and simple words.
    - Use a friendly and approachable tone to make users feel comfortable.
    - If providing instructions with multiple steps, **present only one step at a time**.
    - After each step, ask the user if they were able to complete it or if they are ready to move on to the next step.
      For example:
      - "Did that work for you? Would you like to proceed to the next step?"
    - Do not present multiple points in one response. Only present one point at a time.

    Tracking Previous Queries:
    - Keep track of the user's previous queries to maintain context and continuity throughout the conversation.
    - Refer back to prior questions if relevant, ensuring a seamless experience.
    - Example: "Earlier, you asked about Bluetooth settings. Let’s continue from there."

    Handling Non-Technical or Off-Topic Questions:
    - If the question involves politics, opinions, or unrelated topics (like beliefs or non-technical opinions), respond politely by redirecting the user to technology assistance:
      Example response: "I'm here to help with technology questions. Let’s focus on that!"
    - Avoid engaging in non-technical or personal opinions. Politely guide them back to tech-related assistance.

    Assisting with Basic Technology Questions:
    - If the question is simple and tech-related (e.g., basic device usage or functions), respond clearly with **step-by-step instructions**.
    - Example: For "How do I turn on Bluetooth?":
      - Step 1: "Go to your device's Settings."
      - Step 2: "Select Bluetooth."
      - Step 3: "Switch it to 'On'."
    - Confirm after each step: "Did that help?"

    Handling Advanced Technology Topics:
    - For advanced concepts like AI, machine learning, or technical product details, provide a simplified explanation.
    - Use phrases like "In simple terms" or "To put it plainly" to make answers accessible.
    - Example: "A large language model is a type of AI that helps computers understand and respond to human language."

    Troubleshooting Guidance:
    - For troubleshooting questions, provide **step-by-step guidance**, starting from the simplest solution.
    - Break down each solution into individual steps, confirming with the user after each one:
      - "Let’s go through it step by step. First, try restarting your device… Did that help?"
    - If the problem persists, suggest general help or professional assistance without over-promising a solution.

    Unanswerable Questions or Lack of Context:
    - If you cannot provide a relevant answer, respond with "I don't know."
    - Offer general suggestions like checking online help articles or consulting official resources.
    - Reassure the user: "If I had more details, I’d be glad to help. Here are some general tips…" or "I don’t have that information right now, but you can try…"

    Follow-up Assistance:
    - Always end responses with an offer for further help, like:
      - "Let me know if you have more questions!"
      - "If anything is unclear, I’d be happy to explain further!"

    When responding, do not include any brackets, bold formatting, or any extra notes. Your responses should be direct and user-friendly, without any annotations or explanations about the process.

    Context:
    <context>
    {context}
    </context>

    Previous Queries:
    <previous_queries>
    {previous_queries}
    </previous_queries>

    Question: {input}
    """)
    return prompt


In [32]:
def load_documents(directory):
    global vectorstore, embeddings, model_name, vectorDB_name
    try:
        # Process all PDFs in the directory
        all_chunks = []
        for file in directory:
            print(file)
            if file.name.endswith(".pdf"):
                print(file.name)
                chunks = process_pdf(file.name)
                all_chunks.extend(chunks)

        print(f'number of chunks: {len(all_chunks)}')
        download_embedding_model(model_name)
        # Create FAISS index and vector embeddings for chucks of data
        vectorstore = FAISS.from_documents(all_chunks, embeddings)
        # Save the index
        vectorstore.save_local(vectorDB_name)
        return f"Successfully loaded documents."
    except Exception as e:
        return f"Error loading documents: {str(e)}"

In [ ]:

import gradio as gr
import subprocess
import os
import traceback
from concurrent.futures import ThreadPoolExecutor

# Define the path to the TTS script and the ASR script
base_folder_path = '/content/drive/My Drive/Colab Notebooks/rita/python-clients'
tts_script_path = f"{base_folder_path}/scripts/tts/talk.py"
asr_script_path = f"{base_folder_path}/scripts/asr/transcribe_file.py"
API_KEY = os.getenv("NVIDIA_API_KEY")

# Initialize conversation history
conversation_history = []

# Function to generate audio for the bot's response
def generate_audio_for_response(response_text):
    command = [
        "python3", tts_script_path,
        "--server", "grpc.nvcf.nvidia.com:443",
        "--use-ssl",
        "--metadata", "function-id", "0149dedb-2be8-4195-b9a0-e57e0e14f972",
        "--metadata", "authorization", f"Bearer {API_KEY}",
        "--text", response_text,
        "--voice", "English-US.Female-1",
        "--output", "response_audio.wav"
    ]

    try:
        subprocess.run(command, check=True)
        return "response_audio.wav" if os.path.exists("response_audio.wav") else None
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while generating audio: {e}")
        return None

# Generate welcome audio on startup
welcome_text = "Welcome to Rita, Your Personal Tech Assistant!"
welcome_audio_file = generate_audio_for_response(welcome_text)

# Function to transcribe audio from a file
def transcribe_audio(file_path):
    command = [
        "python3", asr_script_path,
        "--server", "grpc.nvcf.nvidia.com:443",
        "--use-ssl",
        "--metadata", "function-id", "1598d209-5e27-4d3c-8079-4751568b1081",
        "--metadata", "authorization", f"Bearer {API_KEY}",
        "--language-code", "en-US",
        "--input-file", file_path
    ]

    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while transcribing audio: {e}")
        return "Transcription failed."

def chat_response(message, history):
    global vectorstore, embeddings

    llm = ChatNVIDIA(base_url="https://integrate.api.nvidia.com/v1", model="meta/llama-3.2-3b-instruct")
    prompt = prompt_template()

    # Prepare the previous queries as a single string joined by newlines
    previous_queries = "\n".join([f"User: {msg}\nAssistant: {resp}" for msg, resp in history])

    try:
        document_chain = create_stuff_documents_chain(llm, prompt)
        retriever = vectorstore.as_retriever()
        retrieval_chain = create_retrieval_chain(retriever, document_chain)

        # Pass 'previous_queries' along with 'message' and 'context'
        response = retrieval_chain.invoke({
            "input": message,
            "context": "",  # Adjust this as per your context source
            "previous_queries": previous_queries
        })
        return history + [(message, response["answer"])]
    except Exception as e:
        print(traceback.format_exc())
        return history + [(message, f"Error processing query: {str(e)}")]


# Handle audio upload
def handle_uploaded_audio(file):
    global conversation_history
    if file:
        transcription = transcribe_audio(file)
        if transcription:
            conversation_history = chat_response(transcription, conversation_history)
            assistant_response = conversation_history[-1][1]
            audio_file = generate_audio_for_response(assistant_response)
            return "", conversation_history, audio_file
    return "No audio uploaded.", conversation_history, None

# Handle text input
def user_message(text):
    global conversation_history
    if text:
        conversation_history = chat_response(text, conversation_history)
        assistant_response = conversation_history[-1][1]
        audio_file = generate_audio_for_response(assistant_response)
        return "", conversation_history, audio_file

# Gradio interface
with gr.Blocks(css="""
    body {background-color: #ffffff; font-family: Arial, sans-serif;}
    .output-box {background-color: #ffffff; border-radius: 10px; padding: 10px; box-shadow: 0px 4px 6px rgba(0,0,0,0.1);}
    #chatbot {height: 400px; overflow-y: scroll; border-radius: 10px; background-color: #f8f9fa; padding: 10px;}
    .header {text-align: center; margin-bottom: 20px;}
""") as demo:
    gr.Markdown("<div class='header'><h1>👋 Welcome to Rita, Your Personal Tech Assistant!</h1></div>")

    # Audio output box at the top with autoplay enabled
    audio_output = gr.Audio(label="🔊 Assistant's Response", type="filepath", autoplay=True, value=welcome_audio_file)

    chatbot = gr.Chatbot(label="💬 Chatbot Conversation", elem_id="chatbot")
    msg = gr.Textbox(label="✍️ Enter your question", interactive=True, placeholder="Type your message here...")
    ask_rita_btn = gr.Button("Ask Rita")

    with gr.Row():
        audio_input = gr.Audio(label="🎵 Upload Audio for Transcription", type="filepath")

    # Event triggers
    audio_input.change(
        handle_uploaded_audio,
        inputs=[audio_input],
        outputs=[msg, chatbot, audio_output]
    )

    msg.submit(user_message, inputs=[msg], outputs=[msg, chatbot, audio_output])
    ask_rita_btn.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b46b2a307895e444db.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
